In [ ]:
url = ''

## Evaluation Metrics

In [ ]:
def k_precision(true, pred, k=40):
    """
    Compute precision at k
    """
    return len(set(true) & set(pred[:k])) / k

In [ ]:
def mean_average_precision(true, pred, k=40):
    """
    Compute mean average precision at k
    """
    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(pred[:k]):
        if p in true and p not in pred[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not true:
        return 0.0

    return score / min(len(true), k) 

In [ ]:
def r_precision(true, pred):
    """
    Compute precision at r
    """
    return len(set(true) & set(pred)) / len(true)

In [ ]:
def f_measure(true, pred, k=40):
    """
    Compute f-measure at k
    """
    p = k_precision(true, pred, k)
    r = r_precision(true, pred)
    return 2 * p * r / (p + r)

In [ ]:
def evaluate(true, pred, k=40):
    """
    Compute precision, recall and f-measure at k
    """
    p = k_precision(true, pred, k)
    r = r_precision(true, pred)
    f = f_measure(true, pred, k)
    s = mean_average_precision(true, pred, k)

    return f'k_precision: {p}, r_precision: {r}, f_measure: {f}, MAP@K: {s}'

## test queries with MAP

In [ ]:
import json
import requests
import time

In [ ]:
with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [ ]:
query_res = []
pred_wid = {}
total_duration = 0
avg_duration = 0
map_40 = True

for i, wids in enumerate(queries):
    start = time.time()
    try:
        r = requests.get(url + '/search', {'query': i}, timeout=35)
        duration = time.time() - start
        if duration:
            duration_under_35 = True
        total_duration += duration
        avg_duration = total_duration / (i + 1)
        print(f'Query {i} took {duration:.2f} seconds. Average duration: {avg_duration:.2f} seconds.')
        if r.status_code == 200:
            map_40 = mean_average_precision(wids, r.json())
            if map_40  < 0.25:
                map_40 = False
    except:
        pass

print('duration less than 35 seconds:', duration_under_35)
print('map@40:', map_40)
print('average duration:', avg_duration)
        